In [ ]:
class TrafficStressCalculator:
    def segment_lts(self, segment_type, bike_lane_width, parking_lane, speed_limit, num_lanes, adt, has_centerline):
        """
        Calculate the LTS for a segment based on segment type and traffic characteristics.

        :param segment_type: Type of segment ('separated_path', 'mixed_traffic', 'bike_lane_no_parking', 'bike_lane_with_parking').
        :param bike_lane_width: Width of bike lane (meters).
        :param parking_lane: Boolean, if adjacent parking lane exists.
        :param speed_limit: Speed limit of the road (km/h).
        :param num_lanes: Number of traffic lanes.
        :param adt: Average daily traffic volume (ADT).
        :param has_centerline: Boolean, if centerline exists (for mixed traffic).
        :return: LTS level for the segment (1 to 4).
        """
        # Separated Path Segment
        if segment_type == 'separated_path':
            # Separated paths have low LTS, but may be affected by adjacent parking
            return 1 if not parking_lane else 2

        # Mixed Traffic Segment
        elif segment_type == 'mixed_traffic':
            if num_lanes == 1 and not has_centerline:
                # For streets with no centerline and single lane
                if adt <= 750:
                    return 1 if speed_limit <= 46 else 2
                elif adt <= 1500:
                    return 1 if speed_limit <= 38 else 2
                else:
                    return 3 if speed_limit <= 54 else 4
            elif num_lanes >= 2:
                # For streets with two or more lanes
                if speed_limit <= 38:
                    return 2
                elif speed_limit <= 54:
                    return 3 if adt <= 8000 else 4
                else:
                    return 4

        # Bike Lane without Adjacent Parking
        elif segment_type == 'bike_lane_no_parking':
            # Adjustments based on bike lane width and speed limit
            if bike_lane_width >= 1.8:
                return 1 if speed_limit <= 38 else 2 if speed_limit <= 54 else 3
            else:
                return 2 if speed_limit <= 38 else 3 if speed_limit <= 54 else 4

        # Bike Lane with Adjacent Parking
        elif segment_type == 'bike_lane_with_parking':
            # Combined bike lane and parking lane width (reach width)
            reach_width = bike_lane_width + (2.5 if parking_lane else 0)
            if reach_width >= 4.45:
                return 1 if speed_limit <= 46 else 2 if speed_limit <= 62 else 3
            else:
                return 2 if speed_limit <= 46 else 3 if speed_limit <= 62 else 4

        # Default to highest stress level if conditions don't match
        return 4

    def calculate_crossing_lts(self, crossing_type, traffic_speed, crossing_distance, num_traffic_lanes, traffic_flow_direction, bike_lane_pocket):
        """
        Calculate the LTS for a crossing based on various crossing characteristics.

        :param crossing_type: Type of crossing (e.g., 'signalized', 'unsignalized').
        :param traffic_speed: Speed limit of traffic at crossing (km/h).
        :param crossing_distance: Width of crossing area (meters).
        :param num_traffic_lanes: Number of traffic lanes crossing.
        :param traffic_flow_direction: Traffic flow direction indicator (0 = two-way, 1 = one-way).
        :param bike_lane_pocket: Indicates presence of a bike lane pocket (0: none, 1: minimal, 2: moderate, 3: extensive).
        :return: LTS level for the crossing (1 to 4).
        """
        if crossing_type == 'signalized':
            if traffic_speed <= 30:
                return 1 if num_traffic_lanes <= 2 else 2
            elif traffic_speed <= 40:
                return 2 if num_traffic_lanes <= 2 else 3
            else:
                return 3 if num_traffic_lanes <= 2 else 4

        elif crossing_type == 'unsignalized':
            # Higher stress for unsignalized crossings, especially with multiple lanes and high traffic speeds
            if traffic_speed <= 30:
                return 2 if crossing_distance <= 15 else 3
            elif traffic_speed <= 40:
                return 3 if num_traffic_lanes <= 2 else 4
            else:
                return 4

        # Default to highest stress if no conditions matched
        return 4

# Example usage
calc = TrafficStressCalculator()
segment_lts = calc.segment_lts('mixed_traffic', 1.5, False, 50, 2, 1000, True)
print("Segment LTS:", segment_lts)